In [41]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

DF_LEN = 12

def get_table_with_links(driver):
    tiempo_inicial = time.time()
      # Abrir una nueva instancia del browser
    lista_personas_completos = pd.read_csv('Personas.csv')
    personas = lista_personas_completos["Nombres"].unique()
    print(personas)
    url = 'https://appscvs1.supercias.gob.ec/consultaPersona/consulta_cia_param.zul'
    driver.get(url)
    
    for persona in personas:

        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="z-radio"][2]/input'))).click()
            driver.find_element_by_xpath('//span[@class="z-radio"][2]/input').click() #click radio nombre
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-combobox-inp z-combobox-right-edge"]')))
            search_box = driver.find_element_by_xpath('//*[@class="z-combobox-inp z-combobox-right-edge"]')
            #search_box.click()
            #search_box.send_keys('tax')
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-combobox-inp z-combobox-right-edge"]'
        ))).send_keys(persona)
            sleep(2)
            WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//td[@class="z-button-cm"]'))).click() 
            WebDriverWait(driver, 5).until(lambda driver:driver.current_url != url)
            
            admin = []
            acc = []
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-listbox-body"]')))
            df = pd.DataFrame(columns=['Expediente','Administracion', 'Ruc', 'Nacionalidad', 'Cargo', 'Fecha Nombramiento',
                                       'Fecha Termino', 'Periodo', 'Fecha Registro Mercantil', 'Articulo',
                                       '# Registro Mercantil', 'RL/ADM', "Nombre"])
            df1 = pd.DataFrame(columns=['Expediente','Administracion', 'Ruc', 'Nacionalidad', 'Cargo', 'Fecha Nombramiento',
                                       'Fecha Termino', 'Periodo', 'Fecha Registro Mercantil', 'Articulo',
                                       '# Registro Mercantil', 'RL/ADM', "Nombre"])
                                       
            df2 = pd.DataFrame(columns=['Expediente','Empresa', 'Ruc', 'Capital Invertido',
                                        'Capital Total Cia.', 'Valor Nominal', 'Situación Legal', 'Nombre'])  
            df3 = pd.DataFrame(columns=['Expediente','Empresa', 'Ruc', 'Situación Legal', 'Nombre'])  
            for nTable, i in enumerate(driver.find_elements_by_xpath('//*[@class="z-listbox-body"]')):
               
                #print("------------------------------------")
                #print(i)
                #print(i.find_elements_by_xpath('table/tbody/tr/td'))
                #print("------------------------------------")
                for j in i.find_elements_by_xpath('table/tbody/tr'):
                    accionista = []
                    for index, campo in enumerate(j.find_elements_by_xpath('td')):
                        try:
                            accionista.append(campo.text)
                        except:
                            accionista.append("")
                    print("------------------------------------")
                    print(accionista)
                    print(len(accionista))
                    print("------------------------------------")
                    if len(accionista) == DF_LEN and nTable != 2:
                        accionista.append(persona)
                        admin.append(accionista)
                        df.loc[len(df)]=accionista
                    elif len(accionista) == DF_LEN and nTable == 2:
                        accionista.append(persona)
                        admin.append(accionista)
                        df1.loc[len(df1)]=accionista
                    elif len(accionista) == 7:
                        accionista.append(persona)
                        acc.append(accionista)
                        df2.loc[len(df2)]=accionista
                    elif len(accionista) == 4:
                        accionista.append(persona)
                        df3.loc[len(df3)]=accionista
                        
                   
        except:
            print("Persona {} fallido".format(persona))
            df = pd.DataFrame({'Expediente': "",'Administracion':"", 'Ruc':"", 'Nacionalidad':"", 'Cargo':"", 'Fecha Nombramiento':"",
                                       'Fecha Termino':"", 'Periodo':"", 'Fecha Registro Mercantil':"", 'Articulo':"",
                                       '# Registro Mercantil':"", 'RL/ADM':"", 'Nombre':persona}, index=[0])
            df1 = pd.DataFrame({'Expediente': "",'Administracion':"", 'Ruc':"", 'Nacionalidad':"", 'Cargo':"", 'Fecha Nombramiento':"",
                                       'Fecha Termino':"", 'Periodo':"", 'Fecha Registro Mercantil':"", 'Articulo':"",
                                       '# Registro Mercantil':"", 'RL/ADM':"", 'Nombre':persona}, index=[0])
            df2 = pd.DataFrame({'Expediente': "",'Empresa':"", 'Ruc':"", 'Capital Invertido':"",
                                        'Capital Total Cia.':"", 
                                        'Valor Nominal':"", 'Situación Legal':"",
                                        'Nombre':persona}, index=[0])
                                       
            df3 = pd.DataFrame({'Expediente': "",'Empresa':"", 'Ruc':"", 'Situación Legal':"",
                                        'Nombre':persona}, index=[0])
            
            df.to_csv("admin_actual_scrap.csv", index=False, header=False, mode = 'a', sep=";")
            df1.to_csv("admin_anterior_scrap.csv", index=False, header=False, mode = 'a', sep=";")
            df2.to_csv('Accionistas_scrap.csv', index=False, header=False, mode = 'a', sep=";")
            df3.to_csv('Accionistas_Anterior_scrap.csv', index=False, header=False, mode = 'a', sep=";")
            driver.get(url)
            continue
            
        print(admin)
        print(acc)
        df.to_csv("admin_actual_scrap.csv", index=False, header=False, mode = 'a', sep=";")
        df1.to_csv("admin_anterior_scrap.csv", index=False, header=False, mode = 'a', sep=";")
        df2.to_csv('Accionistas_scrap.csv', index=False, header=False, mode = 'a', sep=";")
        df3.to_csv('Accionistas_Anterior_scrap.csv', index=False, header=False, mode = 'a', sep=";")
            
    print("tiempo final:", time.time()-tiempo_inicial)
 
    driver.quit()
    #print(df)

options = Options()
#options.binary_location=r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument("--headless")
my_driver = webdriver.Chrome(options=options)
 
get_table_with_links(my_driver)

['MARTINEZ VELASQUEZ ISIDORO RICARDO' 'CHICAIZA ALVARADO PEDRO GIOVANNY'
 'CONSORCIO CASUARINA' 'CONSORCIO CEMENTERIO SUBURBIO'
 'COMERCIALIZADORA NUTRI MED NUTRIMEDECUADOR S.A.' 'ROCHE ECUADOR S.A.'
 'CAMPOSANTOS DEL ECUADOR S.A. CAMPOECUADOR' 'APOLO APOLO JOSE GONZALO'
 'DIMPROKEL SA' 'CAMACHO JORDAN FULTON ANTONIO']
------------------------------------
[]
0
------------------------------------
------------------------------------
['58710', 'SURREY S.A.', '0991095020001', 'ECUADOR', 'GERENTE GENERAL', '2019-07-01', ' ', '5', '2019-07-16', 'NNN', '1506', 'RL']
12
------------------------------------
------------------------------------
['135114', 'CAROZELI S.A.', '0992631821001', 'ECUADOR', 'GERENTE GENERAL', '2018-11-06', ' ', '5', '2018-11-12', '27', '295', 'RL']
12
------------------------------------
------------------------------------
['132320', 'AFITGER S.A.', '0992589078001', 'ECUADOR', 'PRESIDENTE', '2013-11-27', ' ', '5', '2013-12-11', '27', '3832', 'RL']
12
----------------

Persona APOLO APOLO JOSE GONZALO fallido
Persona DIMPROKEL SA fallido
------------------------------------
[]
0
------------------------------------
------------------------------------
['99675', 'GUSTAVO CAMACHO COMERCIO Y SEGURIDAD S.A.', '0992108290001', 'ECUADOR', 'GERENTE GENERAL', '2019-01-15', ' ', '2', '2019-01-22', '30', '856', 'RL']
12
------------------------------------
------------------------------------
['']
1
------------------------------------
------------------------------------
[]
0
------------------------------------
------------------------------------
['99675', 'GUSTAVO CAMACHO COMERCIO Y SEGURIDAD S.A.', '0992108290001', '80.0', '800', '1.0', 'ACTIVA']
7
------------------------------------
------------------------------------
['']
1
------------------------------------
------------------------------------
[]
0
------------------------------------
------------------------------------
['99675', 'GUSTAVO CAMACHO COMERCIO Y SEGURIDAD S.A.', '0992108290001', 'ECUAD

In [31]:
import pandas as pd
lista_personas_completos = pd.read_csv('Personas.csv',delimiter=';', usecols = [0])

In [3]:
lista_personas_completos.head()

,",Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Nombres"
0,"0,,,,,,,MARTINEZ VELASQUEZ ISIDORO RICARDO"
1,"7,,,,,,,CHICAIZA ALVARADO PEDRO GIOVANNY"
2,"8,,,,,,,CONSORCIO CASUARINA"
3,"11,,,,,,,CONSORCIO CEMENTERIO SUBURBIO"
4,"14,,,,,,,COMERCIALIZADORA NUTRI MED NUTRIMEDEC..."


In [6]:
lista_personas_completos = pd.read_csv('Personas.csv')
personas = lista_personas_completos["Nombres"].unique()
print(personas)

['MARTINEZ VELASQUEZ ISIDORO RICARDO' 'CHICAIZA ALVARADO PEDRO GIOVANNY'
 'CONSORCIO CASUARINA' ...
 'CUMPLIMIENTO A LA CIRCULAR NRO. SERCOP-SERCOP-2020-0018-C DE 20 DE MAYO DE 2020, SUSCRITA POR LA ECO'
 'GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DEL CANTON RIOBAMBA'
 'ACLARATORIA']
